# Players dataframes

En este script nos dedicaremos a crear una base de datos limpia segmentada por bateadores y pitchers. Se exportarán dichas bases de datos contemplando respectivamente a los jugadores que son agentes libres, a los que no son y a todos los jugadores. Las secciones dentro del script son:

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Creación de indicador de si el jugador es agente libre.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [2]:
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 10

In [3]:
print("Para que no nos molesten los mensajes de advertencia.")
import warnings
warnings.filterwarnings('ignore')

Para que no nos molesten los mensajes de advertencia.


In [4]:
free_agents_2011 = 'Data/Free_Agents/Free_Agents_2011.csv'
hitting_2011 = 'Data/Statistics/Hitting/HItting_2011.csv'
pitching_2011 = 'Data/Statistics/Pitching/Pitching_2011.csv'
salary_2011 = 'Data/Salary/Salary_2011.csv'

df_fa_2011 = pd.read_csv(free_agents_2011)
df_h_2011 = pd.read_csv(hitting_2011)
df_p_2011 = pd.read_csv(pitching_2011)
df_s_2011 = pd.read_csv(salary_2011)

## Visualización de las bases de datos

A continuación, se mostrará el contenido de las distintas bases de datos sobre los *bateadores*, *pitchers*, *salarios de los agentes libres* y *salarios de los todos los jugadores*. Esto para determinar el proceso de limpieza que se llevará a cabo.

### Agentes libres

Veamos primero el dataframe

In [9]:
df_fa_2011.head()

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV
0,1,Wil Nieves,2011,C,UFA,WSH,MIL,1,"$775,000","$775,000"


### Hitting

Veamos el dataframe

In [10]:
df_h_2011.head()

,Rank,Player,Pos,Team,GP,GP%,AB,H,HR,RBI,AVG,OPS,Cash2022
0,1,Adrian Gonzalez,1B,BOS,159,0.982,630,213,27,117,0.338,0.957,$0
1,2,Jacoby Ellsbury,CF,BOS,158,0.975,660,212,32,105,0.321,0.928,$0
2,3,Starlin Castro,2B,CHC,158,0.975,674,207,10,66,0.307,0.773,$0
3,4,Melky Cabrera,LF,KC,155,0.957,658,201,18,87,0.306,0.809,$0
4,5,Miguel Cabrera,1B,DET,161,0.994,572,197,30,105,0.344,1.033,"$32,000,000"


Los términos en la base de datos no se traducirán para evitar malentendidos en la traducción.

- **Pos**: Player position.
- **Team**: Team acronym.
- **GP**: Games played.
- **GP%**: Games played %.
- **AB**: At bats.
- **H**: Hitting.
- **HR**: Home runs.
- **RBI**: Runs batted in.
- **AVG**: Batting average.
- **OPS**: Onebase plus slugging%.

Se omitirá la columna *Cash2022* puesto que no es de interés para el trabajo el valor del jugador en la actualidad puesto que hay agentes libres que ya se han retirado en años posteriores.

## Pitching

In [11]:
df_p_2011.head()

,Rank,Player,Pos,Team,GP,GS,IP,H,R,ER,BB,SO,W,L,SV,WHIP,ERA,Cash2022
0,1,Justin Verlander,SP,DET,34,34,251.0,174,73,67,57,250,24,5,0,0.92,2.40,"$25,000,000"
1,2,James Shields,SP,TB,33,33,249.3,195,83,78,65,225,16,12,0,1.04,2.82,$0
2,3,Dan Haren,SP,LAA,35,34,238.3,211,91,84,33,192,16,10,0,1.02,3.17,$0
3,4,C.C. Sabathia,SP,NYY,33,33,237.3,230,87,79,61,230,19,8,0,1.23,3.00,$0
4,5,Jered Weaver,SP,LAA,33,33,235.7,182,65,63,56,198,18,8,0,1.01,2.41,$0


#### Notación.

Veamos a qué se refieren algunos términos

- **Pos**: Player position.
- **Team**: Team acronym.
- **GP**: Games played.
- **GS**: Games started.
- **IP**: Inning pitched.
- **H**: Hits.
- **R**: Runs.
- **ER**: Earned runs.
- **BB**: Walks.
- **SO**: Strikeouts.
- **W**: Wins.
- **L**: Losses-
- **SV**: Saves.
- **WHIP**: WHIP.
- **ERA**: Earned runs average.

Por razones análogas, se descartará la columna *Cash2022*.

### Compensación salarial

En este caso, hay muchas menos variables que en las anteriores bases de datos

In [12]:
df_s_2011.head()

,Rank,Player,Year,Pos,Team,BaseSalary,Payroll Salary,Adj Salary
0,1,Alex Rodriguez,2011,DH,NYY,"$31,000,000","$32,000,000","$32,000,000"
1,2,Vernon Wells,2011,LF,LAA,"$23,000,000","$26,187,500","$18,000,000"
2,3,C.C. Sabathia,2011,SP,NYY,"$24,285,714","$25,571,428","$25,571,428"
3,4,Mark Teixeira,2011,1B,NYY,"$22,500,000","$23,125,000","$23,125,000"
4,5,Joe Mauer,2011,1B,MIN,"$23,000,000","$23,000,000","$23,000,000"


- **BaseSalary**: A base salary is the minimum amount you can expect to earn in exchange for your time or services. This is the amount earned before benefits, bonuses, or compensation is added.
- **Payroll Salary**: Payroll is the compensation a business must pay to its employees for a set period and on a given date.
- **Adj Salary**: Adjusted Salary means the regular salary, wages and commissions, if any, payable to a Participant by the Company for the Participant's service, excluding any bonuses or other compensation.

Nos quedaremos con las columnas de *Player*, *Team* y *Payroll Salary*. Este último se debe a que ya es la compensación salarial neta de cada jugador, por lo que es más significativa que las otras dos columnas referentes al salario.

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [5]:
# Auxiliares:
free_agents = 'Data/Free_Agents/Free_Agents_'
hitting = 'Data/Statistics/Hitting/HItting_'
pitching = 'Data/Statistics/Pitching/Pitching_'
salary = 'Data/Salary/Salary_'
csv = '.csv'
# Originales:
df_free_agents = [None]*10
df_hitting = [None]*10
df_pitching = [None]*10
df_salary = [None]*10
# Copias:
df_free_agents_copy = [None]*10
df_hitting_copy = [None]*10
df_pitching_copy = [None]*10
df_salary_copy = [None]*10
# Producto final:
df_pitchers = [None]*10
df_hitters = [None]*10
df_pitchers_free_agents = [None]*10
df_hitters_free_agents = [None]*10
df_pitchers_no_free_agents = [None]*10
df_hitters_no_free_agents = [None]*10

In [16]:
for i in range(0,10):    
    df_free_agents[i] = pd.read_csv(free_agents + str(2011 + i) + csv)
    df_hitting[i] = pd.read_csv(hitting + str(2011 + i) + csv)
    df_pitching[i] = pd.read_csv(pitching + str(2011 + i) + csv)
    df_salary[i] = pd.read_csv(salary + str(2011 + i) + csv)
    
    df_free_agents_copy[i] = df_free_agents[i].copy()
    df_hitting_copy[i] = df_hitting[i].copy()
    df_pitching_copy[i] = df_pitching[i].copy()
    df_salary_copy[i] = df_salary[i].copy()
    
    df_free_agents_copy[i]  = df_free_agents_copy[i][['Player', 'Value', 'AAV']]
    df_free_agents_names  = ['Jugador', 'Valor_contrato', 'Valor_promedio_contrato']
    df_free_agents_copy[i].columns = df_free_agents_names

    free_agents_aux_1 = df_free_agents_copy[i]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[i]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[i]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[i]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[i]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[i]['Valor_contrato'])
    df_free_agents_copy[i]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[i]['Valor_promedio_contrato'])

    df_hitting_copy[i] = df_hitting_copy[i][['Player', 'Pos', 'GP', 'GP%', 'AB', 'H',
                                             'HR', 'RBI', 'AVG', 'OPS']]
    df_hitting_names = ['Jugador', 'Posicion', 'Juegos', 'Porcetnaje_juegos', 'At-bats',
                        'Bateos', 'Home-runs', 'RBI', 'Porcentaje_bateo', 'OPS']
    df_hitting_copy[i].columns = df_hitting_names
    
    df_pitching_copy[i] = df_pitching_copy[i][['Player', 'Pos', 'GP', 'GS', 'IP', 'H', 
                                               'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV', 
                                               'WHIP', 'ERA']]
    df_pitching_names = ['Jugador', 'Posicion', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
                         'Carreras', 'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                         'Saves', 'WHIP', 'ERA']
    df_pitching_copy[i].columns = df_pitching_names

    df_salary_copy[i] = df_salary_copy[i][['Player', 'Team', 'Payroll Salary']]
    df_salary_names = ['Jugador', 'Equipo', 'Sueldo']
    df_salary_copy[i].columns = df_salary_names
    
    salary_aux_1 = df_salary_copy[i]['Sueldo'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[i]['Sueldo'] = salary_aux_2

    salary_aux_1 = df_salary_copy[i]['Sueldo'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[i]['Sueldo'] = salary_aux_2
    df_salary_copy[i]['Sueldo'] = pd.to_numeric(df_salary_copy[i]['Sueldo'])

    aux_1 = pd.merge(df_free_agents_copy[i], df_hitting_copy[i], on = 'Jugador')
    df_hitters_free_agents[i] = pd.merge(aux_1, df_salary_copy[i], on = 'Jugador')
    
    aux_2 = pd.merge(df_free_agents_copy[i], df_pitching_copy[i], on = 'Jugador')
    df_pitchers_free_agents[i] = pd.merge(aux_2, df_salary_copy[i], on = 'Jugador')
    
    df_hitters[i] = pd.merge(df_hitting_copy[i], df_salary_copy[i], on = 'Jugador')
    df_pitchers[i] = pd.merge(df_pitching_copy[i], df_salary_copy[i], on = 'Jugador')
    
    df_hitters_no_free_agents[i] = df_hitters[i][~df_hitters[i].Jugador.isin(df_hitters_free_agents[i].Jugador)]
    df_pitchers_no_free_agents[i] = df_pitchers[i][~df_pitchers[i].Jugador.isin(df_pitchers_free_agents[i].Jugador)]
    
    # Exportemos los dataframes por separado
    df_hitters_free_agents[i].to_csv('Data/New_Data/Hitters/Free_Agent/free_agents_batters_' + str(2011 + i) + '.csv', index = False)
    df_pitchers_free_agents[i].to_csv('Data/New_Data/Pitchers/Free_Agent/free_agents_pitchers_' + str(2011 + i) + '.csv', index = False)
    df_hitters_no_free_agents[i].to_csv('Data/New_Data/Hitters/No_Free_Agent/no_free_agents_batters_' + str(2011 + i) + '.csv', index = False)
    df_pitchers_no_free_agents[i].to_csv('Data/New_Data/Pitchers/No_Free_Agent/no_free_agents_pitchers_' + str(2011 + i) + '.csv', index = False)

In [17]:
# Algunos ejemplos
df_pitchers_no_free_agents[0].head()

,Jugador,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos,Carreras,Carreras_ganadas,Walks,Strike-outs,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo
0,Justin Verlander,SP,34,34,251.0,174,73,67,57,250,24,5,0,0.92,2.40,DET,13416666
1,James Shields,SP,33,33,249.3,195,83,78,65,225,16,12,0,1.04,2.82,TB,4250000
2,Dan Haren,SP,35,34,238.3,211,91,84,33,192,16,10,0,1.02,3.17,LAA,12750000
3,C.C. Sabathia,SP,33,33,237.3,230,87,79,61,230,19,8,0,1.23,3.00,NYY,25571428
4,Jered Weaver,SP,33,33,235.7,182,65,63,56,198,18,8,0,1.01,2.41,LAA,7365000


In [14]:
df_hitters_no_free_agents[0].head()

,Jugador,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo
0,Adrian Gonzalez,1B,159,0.982,630,213,27,117,0.338,0.957,BOS,6375000
1,Jacoby Ellsbury,CF,158,0.975,660,212,32,105,0.321,0.928,BOS,2500000
2,Starlin Castro,2B,158,0.975,674,207,10,66,0.307,0.773,CHC,440000
3,Melky Cabrera,LF,155,0.957,658,201,18,87,0.306,0.809,KC,1250000
4,Miguel Cabrera,1B,161,0.994,572,197,30,105,0.344,1.033,DET,20000000


In [11]:
df_pitchers_no_free_agents[0]

,Jugador,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos_por_pitch,Carreras,Carreras_ganadas,Walks,Strike-outs,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo
0,Justin Verlander,SP,34,34,251.0,174,73,67,57,250,24,5,0,0.92,2.40,DET,13416666
1,James Shields,SP,33,33,249.3,195,83,78,65,225,16,12,0,1.04,2.82,TB,4250000
2,Dan Haren,SP,35,34,238.3,211,91,84,33,192,16,10,0,1.02,3.17,LAA,12750000
3,C.C. Sabathia,SP,33,33,237.3,230,87,79,61,230,19,8,0,1.23,3.00,NYY,25571428
4,Jered Weaver,SP,33,33,235.7,182,65,63,56,198,18,8,0,1.01,2.41,LAA,7365000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,Scott Kazmir,SP,1,1,1.7,5,5,5,2,0,0,0,0,4.20,27.00,LAA,12000000
322,Rafael Dolis,RP,1,0,1.3,0,0,0,1,1,0,0,0,0.75,0.00,CHC,414000
323,Michael Cuddyer,LF,139,138,1.0,2,0,0,1,0,0,0,0,3.00,0.00,MIN,10525000
324,Skip Schumaker,LF,117,94,1.0,1,2,2,1,2,0,0,0,2.00,18.00,STL,2700000


### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

In [18]:
for i in range(0,10):
    # Condiciones
    condicion_hitter = [df_hitters[i].Jugador.isin(df_hitters_free_agents[i].Jugador)]
    condicion_pitcher = [df_pitchers[i].Jugador.isin(df_pitchers_free_agents[i].Jugador)]
    # Etiquetas
    etiquetas = ['Si']
    
    df_hitters[i]['Agente libre'] = np.select(condicion_hitter, etiquetas, default = 'No')
    df_pitchers[i]['Agente libre'] = np.select(condicion_pitcher, etiquetas, default = 'No')
    
    # Exportemos los dataframes
    df_hitters[i].to_csv('Data/New_Data/Hitters/All_Hitters/hitters_' + str(2011 + i) + '.csv', index = False)
    df_pitchers[i].to_csv('Data/New_Data/Pitchers/All_Pitchers/pitchers_' + str(2011 + i) + '.csv', index = False)

Veamos algunos de los resultados

In [19]:
df_hitters[9].head()

,Jugador,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo,Agente libre
0,Trea Turner,SS,59,0.983,233,78,12,41,0.335,0.982,WSH,7450000,No
1,Marcell Ozuna,LF,60,1.000,228,77,18,56,0.338,1.067,ATL,18000000,Si
2,Jose Abreu,1B,60,1.000,240,76,19,60,0.317,0.987,CHW,12781666,No
3,Freddie Freeman,1B,60,1.000,214,73,13,53,0.341,1.102,ATL,22559375,No
4,D.J. LeMahieu,2B,50,0.833,195,71,10,27,0.364,1.011,NYY,12000000,No


In [20]:
df_pitchers[9].head()

,Jugador,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos,Carreras,Carreras_ganadas,Walks,Strike-outs,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo,Agente libre
0,Lance Lynn,SP,13,13,84.0,64,34,31,25,89,6,3,0,1.06,3.32,TEX,11333333,No
1,German Marquez,SP,13,13,81.7,78,41,34,25,73,4,6,0,1.26,3.75,COL,4800000,No
2,Kyle Hendricks,SP,12,12,81.3,73,26,26,8,64,6,5,0,1.00,2.88,CHC,12000000,No
3,Yu Darvish,SP,12,12,76.0,59,18,17,14,93,8,3,0,0.96,2.01,CHC,22000000,No
4,Brandon Woodruff,SP,13,13,73.7,55,26,25,18,91,3,5,0,0.99,3.05,MIL,633100,No
